In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [3]:
df

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
1,2020-01-04 00:00:41,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
2,2020-01-04 00:01:20,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
3,2020-01-04 00:04:12,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
4,2020-01-04 00:15:19,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,True
...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755411,2022-01-03 23:58:28,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755412,2022-01-03 23:58:46,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755413,2022-01-03 23:59:16,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,True


In [4]:
df_3 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": ("count","sum"),"holiday":"mean","workingday":"mean"}).reset_index()

In [11]:
df_3

,datetime,ges_count,reg_count,holiday,workingday
0,2020-01-04,985,654,0.0,0.0
1,2020-01-05,801,670,0.0,0.0
2,2020-01-06,1349,1229,0.0,1.0
3,2020-01-07,1562,1454,0.0,1.0
4,2020-01-08,1600,1518,0.0,1.0
...,...,...,...,...,...
726,2021-12-30,4231,4024,0.0,1.0
727,2021-12-31,8144,7629,0.0,1.0
728,2022-01-01,5932,5293,0.0,0.0
729,2022-01-02,5182,4392,0.0,0.0


In [6]:
df_3.columns = ["datetime","ges_count","reg_count","holiday","workingday"]

In [16]:
df_3.dtypes

datetime      datetime64[ns]
ges_count              int64
reg_count              Int64
holiday              float64
workingday           float64
dtype: object

In [7]:
df_3["holiday"] = df_3["holiday"].round() 

In [10]:
df_3["workingday"] = df_3["workingday"].round() 

In [12]:
#df_3[(df_3.workingday != 1)&(df_3.workingday != 0)]
df_3.loc[(df_3.holiday != 1)&(df_3.holiday != 0)] 

,datetime,ges_count,reg_count,holiday,workingday


In [ ]:
df[df.datetime.between("2020-2-1", "2020-2-2")]

In [14]:
df_3.holiday.astype(int)

0      0
1      0
2      0
3      0
4      0
      ..
726    0
727    0
728    0
729    0
730    0
Name: holiday, Length: 731, dtype: int32

In [15]:
df_3.workingday.astype(int)

0      0
1      0
2      1
3      1
4      1
      ..
726    1
727    1
728    0
729    0
730    1
Name: workingday, Length: 731, dtype: int32

In [12]:
df_3

,datetime,ges_count,reg_count,holiday,workingday
0,2020-01-04,985,654,0.0,0.0
1,2020-01-05,801,670,0.0,0.0
2,2020-01-06,1349,1229,0.0,1.0
3,2020-01-07,1562,1454,0.0,1.0
4,2020-01-08,1600,1518,0.0,1.0
...,...,...,...,...,...
726,2021-12-30,4231,4024,0.0,1.0
727,2021-12-31,8144,7629,0.0,1.0
728,2022-01-01,5932,5293,0.0,0.0
729,2022-01-02,5182,4392,0.0,0.0


In [18]:
bins = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
df_3['ges_count'] = pd.cut(df_3['ges_count'], bins)

In [20]:
df_slice = pd.crosstab(df_3.holiday,df_3.ges_count)

In [21]:
from scipy.stats import chi2_contingency
#Achtung Die CHiquadrattest übergeben Tabelle muss immer absolute Häufigkeiten entahlten
#darf also keine relativen Häufigkeiten erhatlen

ch2, p, dof, expected = chi2_contingency(df_slice)
print(p)

0.656880454482445


In [16]:
px.scatter(df_3,x="datetime",y="ges_count",color="holiday")

In [ ]:
df_4 = df_3.groupby(pd.Grouper(key=("datetime","holiday[mean]"),freq="QS")).agg({"registered_customer": "mean"}).reset_index()